## Setup

In [3]:
# disabling contracts for speed
import contracts
import yaml
contracts.disable_all()

In [4]:
import duckietown_world as dw

INFO:dt-world:duckietown-world 1.0.30
INFO:zj:zj 2.0.4
INFO:dt-world:contracts 1.8.12 


ha


In [5]:
# reducing the verbosity to critical
dw.logger.setLevel(50)

Utility function to draw in IPython:

In [6]:
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_svg, ipython_draw_html

## Pose interpolation code

In [7]:
import geometry as geo
import numpy as np

In [8]:
def interpolate(q0, q1, alpha):
    v = geo.SE2.algebra_from_group(geo.SE2.multiply(geo.SE2.inverse(q0), q1))
    vi = v * alpha
    q = np.dot(q0, geo.SE2.group_from_algebra(vi))
    return q

## PlacedObject example

All objects in the map are instances of `PlacedObject`. 

To create a new object, subclass `PlacedObject` and implement the drawing method `draw_svg` 
and the `extent_points` method.

The SVG drawing is done using the [`svgwrite`][svgwrite] library.

[svgwrite]: https://svgwrite.readthedocs.io/en/master/

In [9]:
class Person(dw.PlacedObject):

    def __init__(self, radius, *args, **kwargs):
        self.radius = radius
        dw.PlacedObject.__init__(self, *args, **kwargs)
        
    def draw_svg(self, drawing, g):
        # drawing is done using the library svgwrite
        c = drawing.circle(center=(0, 0), r=self.radius, fill='pink')
        g.add(c)
        # draws x,y axes
        dw.draw_axes(drawing, g)

    def extent_points(self):
        # set of points describing the boundary 
        L = self.radius
        return [(-L, -L), (+L, +L)]


## Animation example

Create the interpolated poses:

In [10]:
q0 = geo.SE2_from_translation_angle([0, 0], 0)
q1 = geo.SE2_from_translation_angle([2, -2], np.deg2rad(-90))
print(q1)
# create a sequence of poses 
n = 10
seqs = []
steps = np.linspace(0, 1, num=n)
for alpha in steps:
    q = interpolate(q0, q1, alpha)
    seqs.append(q)


[[ 6.123234e-17  1.000000e+00  2.000000e+00]
 [-1.000000e+00  6.123234e-17 -2.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00]]


Create a root PlacedObject:

In [578]:
root = dw.PlacedObject()
import sys
import traceback





with open("autobot29.yaml", 'r') as stream:

    try:

        data = yaml.safe_load(stream)

    except yaml.YAMLError as exc:

        print(exc)

    

    

    data_points = len(data['trajectory_data'])

    x = np.zeros((data_points,))

    y= np.zeros((data_points,))

    R = np.zeros((3,3, data_points))            
    phi = np.zeros((3, data_points))

    

    dx = 999.999*np.ones((data_points, ))

    dy = 999.999*np.ones((data_points, ))

    dr = 999.999*np.ones((data_points, ))
    dphi = 999.999*np.ones((data_points, ))

    final_trajectory = []

    for idx, [time, traj] in enumerate(data['trajectory_data'].items()):

        x[idx] = np.array(traj[0])

        y[idx] = np.array(traj[1])

        R[:,:,idx] = np.reshape(np.array(traj[3:]), (3,3))
        print(R.shape)
        print(R)
        break
    
        phi[:,idx] = np.array([

                np.arctan2(-R[1,2,idx],R[2,2,idx]), 

                np.arctan2(R[0,2,idx],np.sqrt(R[0,0,idx]**2 + R[0,1,idx]**2)),

                np.arctan2(-R[0,1,idx], R[0,0,idx])])

        
        
        # ROHIT 
                t = np.reshape(np.array(traj[0:3], dtype=np.float64), (3,1))
        R = np.reshape(np.array(traj[3:], dtype=np.float64), (3,3))
        R_temp = np.multiply(R, -1)
        
        t_new = np.matmul(R_temp,t)
        R_new = R.transpose()
        
        print(t_new)
        
        
        x[idx] = np.array(t_new[0])

        y[idx] = np.array(t_new[1])
            
        R[:,:] = R_new
        
        
        phi[:,idx] = np.array([

                np.arctan2(-R[1,2],R[2,2]), 

                np.arctan2(R[0,2],np.sqrt(R[0,0]**2 + R[0,1]**2)),

                np.arctan2(-R[0,1], R[0,0])])
    
        
        #print(phi[:,idx])
        z = phi[2,idx]
        points = np.array([x[idx], y[idx]])
        final_trajectory.append([points, z])
    final_array = final_trajectory
seqs2 = []
for entry in range(0, len(final_array)):
    x = final_array[entry][0][0]
    y = final_array[entry][0][1]
    alpha = final_array[entry][1]
    q5 = geo.SE2_from_translation_angle([x,y],alpha)
    seqs2.append(q5)


(3, 3, 574)
[[[-6.1728e-01  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
    0.0000e+00]
  [ 7.8675e-01  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
    0.0000e+00]
  [-4.6000e-04  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
    0.0000e+00]]

 [[-7.8675e-01  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
    0.0000e+00]
  [-6.1728e-01  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
    0.0000e+00]
  [ 3.3000e-04  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
    0.0000e+00]]

 [[-2.0000e-05  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
    0.0000e+00]
  [ 5.6000e-04  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
    0.0000e+00]
  [ 1.0000e+00  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
    0.0000e+00]]]


Create a SampledSequence of the pose:

In [473]:
timestamps = range(len(seqs2)) # [0, 1, 2, ...]
# SE2Transform is the wrapper for SE2 used by Duckietown World 
transforms = [dw.SE2Transform.from_SE2(_) for _ in seqs2]
seq_me = dw.SampledSequence(timestamps, transforms)


Add the object `me` to the root, saying it's a Person, and with the 
sequence above given as ground truth for the pose.

In [545]:

root.set_object("me", Person(0.5), ground_truth=seq_me)


Finally, draw the animation:

In [501]:
area = dw.RectangularArea((-4, -4), (4, 4))
m = dw.load_map('ethz_amod_lab_k31')
ipython_draw_html(m);


Let's now get the lane object:

In [380]:
from duckietown_world.world_duckietown.tile_template import load_tile_types
from duckietown_world.geo.measurements_utils import iterate_by_class
from duckietown_world.world_duckietown.tile import get_lane_poses
from copy import deepcopy


In [534]:
#records = list(iterate_by_class(m, dw.Tile))
#for recor2 in records:
    #print(recor2.object.kind)

for record in records: # only first 15
    #record = records[20]
    tile = record.object
    
    if tile.kind == 'asphalt':
        continue
    #print(m.children['tilemap'].children.keys())
    template = load_tile_types()[tile.kind]
    tileType = list(template.children.keys())

    if tile.kind == 'straight':
        laneClassesList = tileType
        stringTile = str(laneClassesList[0])
        
    if tile.kind == '3way_left':
        print(tileType)

    else:
        tileTypeDict = template[tileType[0]]
        laneClassesList = list(tileTypeDict.children.keys())  
        template_inner_inner = tileTypeDict[laneClassesList[0]]
        stringTile = str(tileType[0]) + '/' + str(laneClassesList[0])
        
    lane_segment = deepcopy(template[stringTile])
    lane_segment = m.children['tilemap'].children['tile-0-5']
    #lane_segment = lane_segment.children['curve_left'].children['curve'].children['lane1']
    counter = 0
    center_points2 = []
    for timestamp, pose_object in seq_me:
  
        counter += 1
        
        hoi = list(get_lane_poses(m, pose_object.as_SE2()))
        if len(hoi) == 0:
            print(dw.SE2Transform.from_SE2(geo.SE2_from_translation_angle([0,0],0)))
            center_points2.append(dw.SE2Transform.from_SE2(geo.SE2_from_translation_angle([0,0],0)))
            continue
        else:
            #print(hoi[0].lane_pose)
            #print(hoi[0].lane_pose.distance_from_center)
            center_points2.append(hoi[0].lane_pose.center_point)
            print(hoi[0].lane_pose.center_point)
        #lane_pose = lanesegment.lane_pose_from_SE2Transform(pose_object)    
        #center_points.append(lane_pose.center_point)
        
    print(counter)
    break
print(center_points2)

SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([

SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([

SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
SE2Transform([0.0, 0.0],0.0)
574
[SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Transform([0.0, 0.0],0.0), SE2Trans

We can use the function `lane_segment.lane_pose_from_SE2Transform` to get the 
lane pose information (relative heading, etc.), including the projection
to the midlane.

In [569]:
# we now add a marker for projection in the center point
sequence = dw.SampledSequence(seq_me.timestamps, seq_me.center_points2)
m.set_object("projection3", Person(0.2), ground_truth=sequence)
#m.set_object("me", Person(0.1), ground_truth=seq_me)


AttributeError: 'SampledSequence' object has no attribute 'children'

In [559]:
ipython_draw_html(m);

AttributeError: 'list' object has no attribute 'asmatrix2d'

In [288]:
m = dw.load_map('ethz_amod_lab_k31')
sk = dw.get_skeleton_graph(m)

In [571]:
#sequence = dw.SampledSequence(seq_me.timestamps, center_points)
m.set_object("me", Person(0.1), ground_truth=seq_me)
ipython_draw_html(sk.root2);

In [418]:
[k for k, val in pose_object.__dict__.items() if not str(hex(id(val))) in str(val)]

['p', 'theta']

DuckietownMap(children={'tilemap': TileMap(children={'tile-0-5': Tile(children={'curve_left': PlacedObject(children={'curve': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],1.5707963267948966))})}, spatial_relations={'curve': GroundTruth(() -> ('curve',)  SE2Transform([0.0, 0.0],4.71238898038469))})}, spatial_relations={1: GroundTruth(() -> ('curve_left',)  SE2Transform([0.0, 0.0],0.0))}), 'tile-1-5': Tile(children={'straight': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],3.141592653589793))})}, spatial_rel